In [28]:
import torch
import pickle
import numpy as np
import networkx as nx
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool, GINConv, GATConv
from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.loader import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

In [29]:
# Load data splits
# dataset = 'fer2013'
dataset = 'ck'
train_data_path = dataset + '_data/train_data_70_20_10.pkl'
val_data_path = dataset + '_data/val_data_70_20_10.pkl'
test_data_path = dataset + '_data/test_data_70_20_10.pkl'

with open(train_data_path, 'rb') as f:
    train_data = pickle.load(f)
with open(val_data_path, 'rb') as f:
    val_data = pickle.load(f)
with open(test_data_path, 'rb') as f:
    test_data = pickle.load(f)

adjacency_matrix = np.loadtxt('standard_mesh_adj_matrix.csv', delimiter=',')
G = nx.from_numpy_array(adjacency_matrix)

# Add batch attribute to each data object
for data in train_data:
    data.batch = torch.zeros(data.x.size(0), dtype=torch.long)
    data.device = 'cpu'
for data in val_data:
    data.batch = torch.zeros(data.x.size(0), dtype=torch.long)
    data.device = 'cpu'
for data in test_data:
    data.batch = torch.zeros(data.x.size(0), dtype=torch.long)
    data.device = 'cpu'
    
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_loss, model):
        if self.best_score is None:
            self.best_score = val_loss
            self.save_checkpoint(val_loss, model)
        elif val_loss > self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


In [59]:
from torch_geometric.nn import BatchNorm
from torch_geometric.nn import GATConv, GCNConv, SAGEConv, GraphConv

class GIN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device='cpu'):
        super(GIN, self).__init__()
        self.device = device
        nn1 = torch.nn.Sequential(torch.nn.Linear(input_dim, hidden_dim), torch.nn.ReLU(), torch.nn.Linear(hidden_dim, hidden_dim))
        self.conv1 = GINConv(nn1)
        self.bn1 = BatchNorm(hidden_dim)
        
        nn2 = torch.nn.Sequential(torch.nn.Linear(hidden_dim, hidden_dim), torch.nn.ReLU(), torch.nn.Linear(hidden_dim, hidden_dim))
        self.conv2 = GINConv(nn2)
        self.bn2 = BatchNorm(hidden_dim)
        
        self.lin = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data, edge_index, batch):
        data = data.to(self.device)
        x, edge_index, batch = data.to(torch.float), edge_index.to(torch.int64), batch
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)

class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device='cpu'):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=4, concat=True)
        self.conv2 = GATConv(hidden_dim * 4, hidden_dim, heads=4, concat=True)
        self.lin = torch.nn.Linear(hidden_dim * 4, output_dim)
        self.device = device

    def forward(self, data, edge_index, batch):
        data = data.to(self.device)
        x, edge_index, batch = data.to(torch.float), edge_index.to(torch.int64), batch
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)
    
class GCNSAGE(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device='cpu'):
        super(GCNSAGE, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.lin = torch.nn.Linear(hidden_dim, output_dim)
        self.device = device

    def forward(self, data, edge_index, batch):
        data = data.to(self.device)
        x, edge_index, batch = data.to(torch.float), edge_index.to(torch.int64), batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)
    
class GConv(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device='cpu'):
        super(GConv, self).__init__()
        self.conv1 = GraphConv(input_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, hidden_dim)
        self.lin = torch.nn.Linear(hidden_dim, output_dim)
        self.device = device

    def forward(self, data, edge_index, batch):
        data = data.to(self.device)
        x, edge_index, batch = data.to(torch.float), edge_index.to(torch.int64), batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)
    
class GINBNorm(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device='cpu'):
        super(GINBNorm, self).__init__()
        nn1 = torch.nn.Sequential(torch.nn.Linear(input_dim, hidden_dim), torch.nn.ReLU(), torch.nn.Linear(hidden_dim, hidden_dim))
        self.conv1 = GINConv(nn1)
        self.bn1 = BatchNorm(hidden_dim)
        
        nn2 = torch.nn.Sequential(torch.nn.Linear(hidden_dim, hidden_dim), torch.nn.ReLU(), torch.nn.Linear(hidden_dim, hidden_dim))
        self.conv2 = GINConv(nn2)
        self.bn2 = BatchNorm(hidden_dim)
        
        self.lin = torch.nn.Linear(hidden_dim, output_dim)
        self.device = device

    def forward(self, data, edge_index, batch):
        data = data.to(self.device)
        x, edge_index, batch = data.to(torch.float), edge_index.to(torch.int64), batch
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)

In [31]:
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    model.to(device)
    total_loss = 0
    correct = 0
    total = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = out.argmax(dim=1)
        correct += pred.eq(data.y).sum().item()
        total += data.y.size(0)
    return total_loss / len(train_loader), correct / total

def evaluate(model, device, loader, criterion):
    model.eval()
    model.to(device)
    correct = 0
    total = 0
    val_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += pred.eq(data.y).sum().item()
            total += data.y.size(0)
            val_loss += criterion(out, data.y).item()
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())
    return correct / total, val_loss / len(loader), all_labels, all_preds


In [60]:
output_dim = len(np.unique([data.y.item() for data in train_data]))
device = 'cpu'
### CHANGE THE NAME OF THE MODEL HERE
model = GINBNorm(input_dim=3, hidden_dim=64, output_dim=output_dim)
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
batch_size = 32
n_epochs = 5
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
# Initialize model, optimizer, and criterion

# Calculate class weights
label_counts = np.bincount([data.y.item() for data in train_data])
class_weights = 1.0 / label_counts
class_weights = class_weights / class_weights.sum()
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)


for epoch in range(n_epochs):
    train_loss, train_acc = train(model, device, train_loader, optimizer, criterion)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc) 
    test_acc, test_loss, test_labels, test_preds = evaluate(
        model, 
        device, 
        test_loader, 
        criterion
    )
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

In [61]:
explainer = Explainer(
    model=model, 
    algorithm=GNNExplainer(
    ),
    explanation_type="model",
    node_mask_type="attributes",
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='graph',
        return_type='log_probs'
    )
)
# explanations for each node
explanations = []
explanation = explainer(test_data[0].x, test_data[0].edge_index, target=None, batch=test_data[0].batch)
explanations.append(explanation)

In [62]:
explanations[0].node_mask

tensor([[0.2509, 0.3193, 0.2632],
        [0.2688, 0.3090, 0.3346],
        [0.2634, 0.3420, 0.2775],
        ...,
        [0.3139, 0.2656, 0.2618],
        [0.2939, 0.2530, 0.3052],
        [0.5824, 0.2852, 0.2665]])